In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing

In [2]:
fuel1 = MATLIB['UO2'].duplicateMat("fuelFA1")
fuel1.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment
fuel1.set('dens', -10.5216)
fuel1.set('rgb', "238 59 49")

fuel2 = fuel1.duplicateMat("fuelFA2")
fuel2.set('rgb', "255 165 0")

fuel3 = fuel1.duplicateMat("fuelFA3")
fuel3.set('rgb', "255 255 0")

ublank1 = fuel1.duplicateMat("upperfuelBlanketFA1")
ublank1.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
ublank1.set('rgb', "255 255 255")

ublank2 = fuel1.duplicateMat("upperfuelBlanketFA2")
ublank2.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
ublank2.set('rgb', "255 255 255")

ublank3 = fuel1.duplicateMat("upperfuelBlanketFA3")
ublank3.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
ublank3.set('rgb', "255 255 255")

lblank1 = fuel1.duplicateMat("lowerfuelBlanketFA1")
lblank1.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
lblank1.set('rgb', "255 255 255")

lblank2 = fuel1.duplicateMat("lowerBlanketFA2")
lblank2.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
lblank2.set('rgb', "255 255 255")

lblank3 = fuel1.duplicateMat("lowerBlanketFA3")
lblank3.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
lblank3.set('rgb', "255 255 255")


mod1 = MATLIB['H2O'].duplicateMat("modFA1")
mod1.set('rgb', "198 226 255")
mod2 = MATLIB['H2O'].duplicateMat("modFA2")
mod2.set('rgb', "198 226 255")
mod3 = MATLIB['H2O'].duplicateMat("modFA3")
mod3.set('rgb', "198 226 255")
thermalShield = MATLIB['H2O'].duplicateMat("thermalShield_water")
thermalShield.set('rgb', "198 226 255")

clad = MATLIB['Zr'].duplicateMat("clad")
clad.set('rgb', "183 183 183")

barrel = MATLIB['Zr'].duplicateMat("barrel_ss")
barrel.set('rgb', "183 183 183")

vessel = MATLIB['Zr'].duplicateMat("vessel_ss")
vessel.set('rgb', "183 183 183")

ref = MATLIB['Reflector'].duplicateMat("reflector") # reflector plenum
ref.set('rgb', "204 153 255")

ba = MATLIB['B'].duplicateMat("B4C") # 90% wt enriched b-10

MB = 10.8
MC = 12
MB4C = MB*4 + MC
rhoB4C = 2.52
AvroNorm = .6022
nb4c = rhoB4C*AvroNorm/MB4C
nc12 = nb4c
b10_enrich = .90
b11_enrich = 1 - b10_enrich
nb10 = nb4c*b10_enrich
nb11 = nb4c*b11_enrich

b4cdens = -2.52
ba.set('rgb', "255 255 0")
ba.set('nuclides', np.array([5010, 5011, 6012]))
ba.set('fractions', np.array([nb10, nb11, nc12]))
ba.set('dens', -1*rhoB4C)
ba.set('isBurn', True)

In [3]:
RT = pin("RT", 1) #top reflector
RT.set('materials', [ref])

RB = pin("RB", 1) #bottom reflector
RB.set('materials', [ref])

In [4]:
ro_fuel = .4096
ro_clad = .4760
ri_gt = .5531
ro_gt = .6133

In [7]:
F1 = pin("F1", 3)
F1.set('materials', [fuel1, clad, mod1])
F1.set('radii', [ro_fuel, ro_clad])


In [24]:
axialheights = np.array([
    0.000000, 
    20.00000,
    25.00000,
    25.00000 + 10.2777777778*1, 
    25.00000 + 10.2777777778*2,
    25.00000 + 10.2777777778*3,
    25.00000 + 10.2777777778*4,
    25.00000 + 10.2777777778*5,
    25.00000 + 10.2777777778*6,
    25.00000 + 10.2777777778*7,
    25.00000 + 10.2777777778*8,
    25.00000 + 10.2777777778*9,
    25.00000 + 10.2777777778*10,
    25.00000 + 10.2777777778*11,
    25.00000 + 10.2777777778*12,
    25.00000 + 10.2777777778*13,
    25.00000 + 10.2777777778*14,
    25.00000 + 10.2777777778*15,
    25.00000 + 10.2777777778*16,
    25.00000 + 10.2777777778*17,
    210.0000,
    220.0000
])

In [84]:
def discretize(pin2D, heights, temps = None):
    pinStackMats = [[0]*len(pin2D.materials)]*len(heights)
    for i in range(0, len(pinStackMats)):
        for j in range(0, len(pinStackMats[0])):
            pinStackMats[i][j] = pin2D.materials[j].duplicateMat(pin2D.materials[j].id+"_z"+str(i))
    pinStackPins = list(np.zeros(len(heights)))
    for i in range(0, len(heights)):
        pinStackPins[i] = pin2D.duplicate(pin2D.id+"_z"+str(i))
        pinStackPins[i].set('materials', pinStackMats[i])
    
    pinStackPins = np.array(pinStackPins)
        
    pinstack3D = pinStack(pin2D.id, 0, 0, len(heights))
    pinstack3D.setStack(pinStackPins, heights)
    return pinstack3D

In [86]:
F1_3D = discretize(F1, axialheights)
print(F1_3D.toString())

lat F1 9 0 0 22
0.0	F1_z0
20.0	F1_z1
25.0	F1_z2
35.2777777778	F1_z3
45.5555555556	F1_z4
55.833333333400006	F1_z5
66.1111111112	F1_z6
76.388888889	F1_z7
86.66666666680001	F1_z8
96.94444444460001	F1_z9
107.2222222224	F1_z10
117.5000000002	F1_z11
127.777777778	F1_z12
138.05555555580003	F1_z13
148.33333333360002	F1_z14
158.61111111140002	F1_z15
168.88888888920002	F1_z16
179.16666666700002	F1_z17
189.4444444448	F1_z18
199.7222222226	F1_z19
210.0	F1_z20
220.0	F1_z21

pin F1_z0
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z1
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z2
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z3
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z4
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z5
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z6
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z7
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z8
fuelFA1_z21	0.4096
clad_z21	0.476
modFA1_z21

pin F1_z9
fuelFA1_z21	0.4096
c

In [87]:
print(F1_3D.univMats)

{'F1_z0': {'fuelFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130288>, 'clad_z21': <serpentGenerator.functions.material.material object at 0x0000015129130348>, 'modFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130C08>}, 'F1_z1': {'fuelFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130288>, 'clad_z21': <serpentGenerator.functions.material.material object at 0x0000015129130348>, 'modFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130C08>}, 'F1_z2': {'fuelFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130288>, 'clad_z21': <serpentGenerator.functions.material.material object at 0x0000015129130348>, 'modFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130C08>}, 'F1_z3': {'fuelFA1_z21': <serpentGenerator.functions.material.material object at 0x0000015129130288>, 'clad_z21': <serpentGenerator.functions.material